In [ ]:
#load model package
#!pip3 install pytorch_lightning

In [2]:
#load attention map package
import os

#!pip install bytecode
!git clone https://github.com/luo3300612/Visualizer.git
# os.chdir('/HoopPad/Visualizer/')
!python setup.py install
from visualizer import get_local
get_local.activate()

Cloning into 'Visualizer'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (240/240), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 240 (delta 37), reused 225 (delta 28), pack-reused 0
Receiving objects: 100% (240/240), 3.83 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (37/37), done.
zsh:1: command not found: python


ModuleNotFoundError: No module named 'visualizer'

In [7]:
import os
import glob
os.chdir('/')

In [11]:
import sys
sys.path.append('/Users/yufu/Documents/Code/HoopPad/vis')
from Ball import Ball
from Player_v import Player_v
from Moment import Moment
from Constant import Constant
import numpy as np
from PIL import Image,ImageFont,ImageDraw
import cv2
import imageio
def draw_cycle(player,radius,court_side):
    if court_side:
        center_x = (94 - player.x) * 10
        center_y = (50 - player.y) * 10
    else:
        center_x = player.x * 10
        center_y = player.y * 10
    #center_x = np.random.normal(scale=0.5) + center_x
    #center_x = np.random.normal(scale=0.5) + center_x
    x1, y1 = center_x - radius, center_y - radius
    x2, y2 = center_x + radius, center_y + radius

    return((x1,y1,x2,y2))



def create_gif(scene,sample_size):

    game_id = scene.gameid
    event_id = scene.id
    moments = [Moment(moment) for moment in scene.moments]


    image = Image.open('/Users/yufu/Documents/Code/HoopPad/vis/half.jpg')
    draw = ImageDraw.Draw(image)
    radius = 10
    teamid = scene.off_teamid
    court_side = moments[0].ball.x > 47.0

    color = (0, 0, 0)
    font_path = os.path.join(cv2.__path__[0],'qt','fonts','DejaVuSans.ttf')
    font = ImageFont.truetype(font_path, size=18)

    frames = []
    frame_num = 1

    for moment in moments[::sample_size]:
        frame_img = Image.open('/Users/yufu/Documents/Code/HoopPad/vis/half.jpg')

        # 创建新的ImageDraw对象
        frame_draw = ImageDraw.Draw(frame_img)
        player_num = 0
        for player in moment.players:
            if player.team.id == teamid:
                x1, y1, x2, y2 = draw_cycle(player,10,court_side )
                frame_draw.ellipse([(x1, y1), (x2, y2)], fill=(255, 0, 0))
                player_num = player_num + 1
                text = str(player_num)  # 您要写的数字
                text_bbox = draw.textbbox((x1, y1), text, font=font)

                # 提取文本的宽度和高度
                text_width = text_bbox[2] - text_bbox[0]
                #text_height = text_bbox[3] - text_bbox[1]

                # 计算文本的位置，以确保文本位于圆的中心
                text_x = x1 + radius - text_width // 2
                #text_y = y1 + radius - text_height // 2

                frame_draw.text((text_x, y1), text, fill=color, font=font)
            else:
                x1, y1, x2, y2 = draw_cycle(player,10,court_side )
                frame_draw.ellipse([(x1, y1), (x2, y2)], fill=(0, 0, 255))
                player_num = player_num +1
                text = str(player_num)
                text_bbox = draw.textbbox((x1, y1), text, font=font)

                # 提取文本的宽度和高度
                text_width = text_bbox[2] - text_bbox[0]
                #text_height = text_bbox[3] - text_bbox[1]

                # 计算文本的位置，以确保文本位于圆的中心
                text_x = x1 + radius - text_width // 2
                #text_y = y1 + radius - text_height // 2
                frame_draw.text((text_x, y1), text, fill=(255, 255, 255), font=font)



        ball = moment.ball
        frame_draw.ellipse(draw_cycle(ball,10,court_side ), fill=(0, 255, 0))

        #标记第几帧
        text = 'frame:'+ str(frame_num)
        text_bbox = draw.textbbox((300, 25), text, font=font)
        frame_draw.text((300, 25), text, fill=color, font=font)

        frames.append(frame_img.copy())
        frame_num +=1

    #save_name = play_name + '/'+ str(game_id) + '_' + str(event_id) + '_'+ str(frame_num) + '.mp4'
    save_name = str(game_id) + '_' + str(event_id) + '_'+ str(frame_num) + '.mp4'
    print(save_name)
    #frames[0].save(save_name, format='GIF', append_images=frames[1:], save_all=True, duration=0.2, loop=0)
    fps = 15  # 帧率
    imageio.mimsave(save_name, frames, format='mp4', fps=fps)
    frame_img.close()

ModuleNotFoundError: No module named 'cv2'

In [6]:
import os
import glob
root_dir = 'HoopPad/Data'
filelist = glob.glob(os.path.join(root_dir, '*.pkl'), recursive=True)
#filelist = [os.path.join(root_dir, f) for f in os.listdir(root_dir)]
len(filelist)

In [1]:
import sys


import pickle
import math
import numpy as np
import pandas as pd
from Model.possession import *
scene_list = []
for filepath in filelist:
    with open(filepath, 'rb') as f:
        scene = pickle.load(f)
        if len(scene.time_24s) >= 150:
            scene_list.append(scene)
len(scene_list)

NameError: name 'filelist' is not defined

In [10]:
# To do add player embedding
def get_newid(playerid,df):
    if playerid in df['playerid'].values:
        newid = df[df['playerid'] == playerid]['UnifiedPlayerID'].values[0]
    else:
        newid = df['UnifiedPlayerID'].max() + 1
        df.loc[len(df)] = {'playerid': playerid, 'UnifiedPlayerID': newid}

    return newid

In [11]:
scene = scene_list[134]
batch = [scene]


### **Generate defense according to offense**







In [14]:
def create_def(batch): #batch代表n个scene

    sample_freq = 5
    sequence_length = int(24*25/sample_freq + 1)
    list_24s = [i*(1/sample_freq) for i in range(0, sequence_length)]
    list_24s.reverse()

    time_steps = sequence_length
    states_batch = np.array([]).reshape(-1,time_steps,3)
    states_padding_batch = np.array([]).reshape(-1,time_steps)
    states_hidden_BP_batch = np.array([]).reshape(-1,time_steps)
    num_agents = np.array([])

    agent_ids_batch = np.array([]) #每一个batch的agent ids, = batch size * 11
    team_ids_batch = np.array([])

    player_newids = pd.read_csv('/content/drive/MyDrive/EPV/DataLoader/players.csv')
    for scene in batch:
        scene_tensor = None
        agent_ids = np.array([])
        team_ids = np.array([])
        for i in range(len(scene.agents)):
            agent = scene.agents[i]
            agent_id = get_newid(agent.playerid,player_newids)
            agent_ids = np.append(agent_ids,agent_id)
            #x = np.array(agent.x)[::sample_freq] #间隔sample_freq个取值
            #y = np.array(agent.y)[::sample_freq] #间隔sample_freq个取值
            #v = np.array(agent.v)[::sample_freq] #间隔sample_freq个取值
            #single_agent_tensor = torch.Tensor([[x,y,v]])

            agent_team = agent.teamid

            if agent_team == -1:
                team_ids= np.append(team_ids,0)
            elif agent_team == scene.off_teamid:
                team_ids = np.append(team_ids,1)
            else:
                team_ids = np.append(team_ids,2)

            single_agent_tensor = torch.Tensor([[x,y,v] for x,y,v in zip(agent.x,agent.y, agent.v)][::sample_freq])
            single_agent_tensor = single_agent_tensor.permute(1, 0)
            single_agent_tensor = torch.unsqueeze(single_agent_tensor, 0)
            single_agent_tensor = single_agent_tensor.to(torch.float)

            if scene_tensor == None:
                scene_tensor = single_agent_tensor
            else:
                scene_tensor = torch.cat([scene_tensor, single_agent_tensor], dim=0) #获得6个agent的tensor shape=[6,3,time_steps]
        #补齐A的维度为6
        A,D,T = scene_tensor.size()

        #补齐A的维度为11
        A,D,T = scene_tensor.size()
        # 如果agents 不足11个：用-1 补齐scence_tensor,agent_ids也用-1补齐
        if A < 11:
            new_dim = torch.full((11-A, D, T), -1)
            scene_tensor = torch.cat([new_dim, scene_tensor], dim=0)
            agent_ids = np.pad(agent_ids, (11 - A, 0), 'constant', constant_values=(-1,))
            team_ids = np.pad(team_ids, (11 - A, 0), 'constant', constant_values=(-1,))


        scene_tensor = torch.transpose(scene_tensor , dim0=1, dim1=2) #shape[A,T,D]

        time_24s = scene.time_24s[::sample_freq] #间隔sample_freq个取值
        if len(time_24s) < 30:
            continue

        end_padding_size =  121 - len(time_24s) #endtime～0

        states_feat = torch.nn.functional.pad(scene_tensor,  (0, 0, 1, end_padding_size-1, 0, 0), mode='constant', value=-1) #[6,121,3]
        states_padding = states_feat[:,:,0]
        states_padding = states_padding < 0 #bool型，[6,121] True为padding，ignore

        team_ids_batch = np.append(team_ids_batch,team_ids) # team_classification for task 3 (0-ball,1-off,2-def)
        def_index = np.where(team_ids_batch == 2)[0]

        states_hidden = np.zeros((11,121)).astype(np.bool_) # True 为hidden，这里不需要hidden,all false
        states_hidden[def_index,5:(T+1)] = True
        agent_ids_batch = np.append(agent_ids_batch,agent_ids) #player_embedding


        num_agents = np.append(num_agents, len(states_feat)) # numpy array(batch_size,) [11,11,11,11,11] #show which row is from same possession

        states_batch = np.concatenate((states_batch,states_feat), axis=0)
        states_padding_batch = np.concatenate((states_padding_batch,states_padding), axis=0)
        states_hidden_BP_batch = np.concatenate((states_hidden_BP_batch,states_hidden), axis=0)

    num_agents_accum = np.cumsum(np.insert(num_agents,0,0)).astype(np.int64)
    agents_batch_mask = np.ones((num_agents_accum[-1],num_agents_accum[-1])) #0代表一个回合内，需要attention

    for i in range(len(num_agents)):
        agents_batch_mask[num_agents_accum[i]:num_agents_accum[i+1], num_agents_accum[i]:num_agents_accum[i+1]] = 0

    states_batch = torch.FloatTensor(states_batch)
    agents_batch_mask = torch.BoolTensor(agents_batch_mask)
    states_padding_batch = torch.BoolTensor(states_padding_batch)
    states_hidden_BP_batch = torch.BoolTensor(states_hidden_BP_batch)

    agent_ids_batch = torch.FloatTensor(agent_ids_batch)
    team_ids_batch = torch.FloatTensor(team_ids_batch)
    labels_batch = torch.FloatTensor(team_ids_batch)


    return (states_batch, agents_batch_mask, states_padding_batch, states_hidden_BP_batch,num_agents_accum,agent_ids_batch,team_ids_batch,labels_batch)


In [16]:
states_batch, agents_batch_mask, states_padding_batch, states_hidden_BP_batch,num_agents_accum,agent_ids_batch,team_ids_batch,labels_batch = create_def(batch)

In [28]:
#load model parameters
import torch

from Model.module_all import HoopTransformer

Scene_model = HoopTransformer(3,121,256,4,4,1,50,0.1,32,5e-5,[1])

from Model.motion_module import *
base_model = test_model_motion(Scene_model,num_unfreeze = 0)

Test_model = Scene_Motion(model=base_model,lr=1e-3)

ckp_path = '/Checkpoints/V2_prompt.ckpt'

checkpoint = torch.load(ckp_path, map_location=lambda storage, loc: storage)
Test_model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [31]:
Test_model.to('cpu')
Test_model.eval()

get_local.clear()

with torch.no_grad():
    out = Test_model(states_batch, agents_batch_mask, states_padding_batch, states_hidden_BP_batch,agent_ids_batch,team_ids_batch)

out.shape
F = out[:,:,0,::2]

In [ ]:
# create attention map
cache = get_local.cache
print(list(cache.keys()))
attention_maps = cache['SelfAttLayer_Dec.forward']
attention_maps[3].shape

att_map_3 = attention_maps[3]#F,timestep,agents,attention

att_map_1 = attention_maps[1] #F,timestep,agents,attention

matrix = att_map_3[15,0:6,:-1]
plt.imshow(matrix, cmap='hot', interpolation='nearest')

['SelfAttLayer_Enc.forward', 'SelfAttLayer_Dec.forward']


In [26]:
def create_gif_ghost(scene,sample_size,F,version):

    game_id = scene.gameid
    event_id = scene.id
    moments = [Moment(moment) for moment in scene.moments]


    image = Image.open('/Users/yufu/Documents/Code/HoopPad/vis/half.jpg').convert("RGBA")
    draw = ImageDraw.Draw(image)
    radius = 10
    teamid = scene.off_teamid
    court_side = moments[0].ball.x > 47.0

    color = (0, 0, 0,255)
    font_path = os.path.join(cv2.__path__[0],'qt','fonts','DejaVuSans.ttf')
    font = ImageFont.truetype(font_path, size=18)

    frames = []
    frame_num = 1

    moments_ = moments[::sample_size]

    for moment in moments_:
        frame_img = Image.open('/Users/yufu/Documents/Code/HoopPad/vis/half.jpg').convert("RGBA")

        # 创建新的ImageDraw对象
        frame_draw = ImageDraw.Draw(frame_img)
        player_num = 0

        for player in moment.players:
            if player.team.id == teamid:
                x1, y1, x2, y2 = draw_cycle(player,10,court_side )
                frame_draw.ellipse([(x1, y1), (x2, y2)], fill=(255, 0, 0,255))
                player_num = player_num + 1
                text = str(player_num)  # 您要写的数字
                text_bbox = draw.textbbox((x1, y1), text, font=font)

                # 提取文本的宽度和高度
                text_width = text_bbox[2] - text_bbox[0]
                #text_height = text_bbox[3] - text_bbox[1]

                # 计算文本的位置，以确保文本位于圆的中心
                text_x = x1 + radius - text_width // 2
                #text_y = y1 + radius - text_height // 2

                frame_draw.text((text_x, y1), text, fill=color, font=font)
            else:
                x1, y1, x2, y2 = draw_cycle(player,10,court_side )
                frame_draw.ellipse([(x1, y1), (x2, y2)], fill=(0, 0, 255,255))
                player_num = player_num +1
                text = str(player_num)
                text_bbox = draw.textbbox((x1, y1), text, font=font)

                # 提取文本的宽度和高度
                text_width = text_bbox[2] - text_bbox[0]
                #text_height = text_bbox[3] - text_bbox[1]

                # 计算文本的位置，以确保文本位于圆的中心
                text_x = x1 + radius - text_width // 2
                #text_y = y1 + radius - text_height // 2
                frame_draw.text((text_x, y1), text, fill=(255, 255, 255,255), font=font)

                if frame_num >= 5:
                    x1 = F[player_num,frame_num,0]*10 - 10
                    x2 = F[player_num,frame_num,0]*10 + 10
                    y1 = F[player_num,frame_num,1]*10 - 10
                    y2 = F[player_num,frame_num,1]*10 + 10
                    frame_draw.ellipse([(x1, y1), (x2, y2)], fill=(0, 255, 255,77))
                    text = str(player_num)
                    text_bbox = draw.textbbox((x1, y1), text, font=font)

                    text_width = text_bbox[2] - text_bbox[0]
                    #text_height = text_bbox[3] - text_bbox[1]

                    # 计算文本的位置，以确保文本位于圆的中心
                    text_x = x1 + radius - text_width // 2
                    #text_y = y1 + radius - text_height // 2
                    frame_draw.text((text_x, y1), text, fill=(255, 255, 255,77), font=font)



        ball = moment.ball
        frame_draw.ellipse(draw_cycle(ball,10,court_side ), fill=(0, 255, 0,255))

        #标记第几帧
        text = 'frame:'+ str(frame_num)
        text_bbox = draw.textbbox((300, 25), text, font=font)
        frame_draw.text((300, 25), text, fill=color, font=font)

        frames.append(frame_img.copy())
        frame_num +=1

    #save_name = play_name + '/'+ str(game_id) + '_' + str(event_id) + '_'+ str(frame_num) + '.mp4'
    save_name = str(game_id) + '_' + str(event_id) + '_'+ str(frame_num) + str(version)+'.mp4'
    print(save_name)
    #frames[0].save(save_name, format='GIF', append_images=frames[1:], save_all=True, duration=0.2, loop=0)
    fps = 2  # 帧率
    imageio.mimsave(save_name, frames, format='mp4', fps=fps)


    frame_img.close()

In [32]:
create_gif_ghost(scene,5,F,'V1')

0021500356_11_80V1.mp4


Add prompt

In [ ]:
#让1和3在53帧的时候在原来的位置
states_hidden_BP_batch[1,53] = False
states_hidden_BP_batch[3,53] = False
states_hidden_BP_batch[2,57] = False
#让1，5在 61-66换位置（换人）
pos = states_batch[1,61:66,:]
states_batch[1,61:66,:] = states_batch[5,61:66,:]
states_batch[5,61:66,:] = pos
states_hidden_BP_batch[1,61:66] = False
states_hidden_BP_batch[5,61:66] = False

In [ ]:
Test_model.to('cpu')
Test_model.eval()

get_local.clear()

with torch.no_grad():
    out = Test_model(states_batch, agents_batch_mask, states_padding_batch, states_hidden_BP_batch,agent_ids_batch,team_ids_batch)

out.shape
F = out[:,:,0,::2]

In [ ]:
create_gif_ghost(scene,5,F,'Prompt')

0021500356_11promptprompt_2prompt_2prompt_2_803.mp4
